In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xsensdeviceapi as xda
import sys
from threading import Lock
import re
import time
import threading as th
import os

In [2]:
def stopAll():
    global device
    global control
    global Ports
    print("Arrêt de l'enregistrement, mise du dispositif en mode configuration.")
    device.stopRecording()
    device.gotoConfig()
    device.disableRadio()
    print("Fermeture du fichier mtb")
    device.closeLogFile()
    print("Fermeture du port")
    control.closePort(Ports)
    control.close()
    xda.XsControl.destruct(control)
    
def checkConnectedSensors(devIdAll):
    global children
    global control
    global device
    childUsed = np.full(np.shape(children),False).tolist()
    if children.empty():
        print("Aucun dispositif trouvé.")
        stopAll()
        raise RuntimeError("Échec. Aucun capteur MTw trouvé.")
    else:
        for i in range(len(children)):
            if children[i].connectivityState() == xda.XCS_Wireless:
                childUsed[i] = True
        
        print("Dispositifs rejetés:")
        rejects = np.array(devIdAll)[[not elem for elem in childUsed]].tolist()
        for i in range(len(rejects)):
            index = devIdAll.index(rejects[i])
            print("%d - "%index +"%s" %rejects[i])
        
        print("Dispositifs acceptés:")
        accepted = np.array(devIdAll)[childUsed].tolist()
        for i in range(len(accepted)):
            index = devIdAll.index(accepted[i])
            print("%d - "%index +"%s" %accepted[i])
        
        option = str(input('Keep current status?'+' (y/n): ')).lower().strip()
        change = []
        if option[0] == 'n':
            op2 = input("\n Type the numbers of the sensors (csv list, e.g. 1,2,3) from which status should be changed \n (if accepted than reject or the other way around):\n")
            change = [int(i) for i in re.split(",",op2)]
            for i in range(len(change)):
                if a[change[i]]:
                    device.rejectConnection(children[change[i]])
                    childUsed[change[i]] = False
                else:
                    device.acceptConnection(children[change[i]])
                    childUsed[change[i]] = True
        if sum(childUsed) == 0:
            stopAll()
            raise RuntimeError("Échec. Aucun capteur MTw trouvé.")
    devicesUsed = np.array(children)[childUsed].tolist()
    devIdUsed = np.array(devIdAll)[childUsed].tolist()
    nDevs = sum(childUsed)

    return devicesUsed, devIdUsed, nDevs

## Thread to stop recording loop ##
def key_capture_thread():
    global keep_going
    input("Press enter to stop recording.")
    keep_going = False
###################################


## Thread to read data on each MTw ##
def childCount(index):
    global matrix_nDev, acc_nDev, pack_nDev, contCalData_nDev, frame_nDev, counter_nDev, tim_nDev
    global keep_going 
    while keep_going:
        pack_nDev[index].append(children[index].getDataPacketCount())
        packet = children[index].getDataPacketByIndex(pack_nDev[index][-1])
        if packet.containsCalibratedData():
            contCalData_nDev[index].append(packet.containsCalibratedData())
            tim_nDev[index].append(packet.utcTime())
            frame_nDev[index].append(packet.frameRange())
            counter_nDev[index].append(packet.packetCounter())
            acc_nDev[index].append(packet.calibratedAcceleration())
            matrix_nDev[index].append(packet.orientationMatrix())

######################################

In [3]:
try:
    # Extraire la version du Xs Device Api employé

    print("Creation d'un objet XsControl...")
    control = xda.XsControl_construct()
    assert(control is not 0)

    xdaVersion = xda.XsVersion()
    xda.xdaVersion(xdaVersion)
    print("Utilisation de la version XDA: %s \n" % xdaVersion.toXsString())


    print("Détection des ports#...\n")
    ports_Scan = xda.XsScanner_scanPorts(0,100,True,True)

    # Détection des dispositifs MTw ou Station Awinda
    Ports = xda.XsPortInfo()
    for i in range(ports_Scan.size()):
        if ports_Scan[i].deviceId().isWirelessMaster() or ports_Scan[i].deviceId().isAwindaXStation():
            
            Ports = ports_Scan[i]
            break

    if Ports.empty():
        raise RuntimeError("Échec. Aucun dispositif a été detecté.")

    devId = Ports.deviceId()
    print(" Dispositif trouvé avec les caractéristiques suivantes:")
    print(" Device ID: %s" % devId.toXsString())
    print(" Nom du port: %s" % Ports.portName())
    print(" Port baudrate: %s" % Ports.baudrate())

    print("Ouverture du port...")
    if not control.openPort(Ports.portName(), Ports.baudrate()):
        raise RuntimeError("Échec de l'ouverture du port.")

    device = control.device(devId)

    assert(device is not 0)
    print("Dispositif: %s, avec ID: %s ouvert. \n" %(device.productCode(), device.deviceId().toXsString()))

   
    # Mettre le dispositif en mode configuration
    print("Dispositif en mode configuration...\n")
    if not device.gotoConfig():
        raise RuntimeError("Échec. Le dispositif n'a as pu être mis en mode configuration.")

    device.setOptions(xda.XSO_Orientation + xda.XSO_Calibrate + xda.XSO_RetainLiveData, 0)


    # Acquisition des fréquences pour la mise à jour des données
    fq_support = device.supportedUpdateRates(xda.XDI_None)
    index_fq = []

    while index_fq == []:
        print("Les fréquences disponibles sont:")
        print('\n'.join('{}: {}'.format(*k) for k in enumerate(fq_support)))
        choix_fq = input("Veuillez choisir une fréquence d'acquisition.")
        if choix_fq == []:
            continue
        index_fq = fq_support.index(int(choix_fq))
    print("Fréquence d'acquisition choisi: ", fq_support[index_fq], "\n")

    # Mise à jour de la fréquence d'acquisition
    if not device.setUpdateRate(fq_support[index_fq]):
        raise RuntimeError("Échec de la mise à jour du update rate.")

    
    # Acquisition du canal radio
    canaux_radio = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
    index_radio = []

    while index_radio == []:
        print("Les canaux disponibles sont: ")
        print('\n'.join('{}: {}'.format(*k) for k in enumerate(canaux_radio)))
        choix_radio = input("Veuillez choisir un canal de radio.")
        if choix_radio == []:
            continue
        index_radio = canaux_radio.index(int(choix_radio))
    print("Canal de radio choisi: ", canaux_radio[index_radio], "\n")

    # Mise à jour du canal radio
    try:
        device.enableRadio(canaux_radio[index_radio])
    except ValueError:
        print("La radio est encore activée, veuillez débrancher l'appareil de l'ordinateur et reessayer.")


    input('\n Undock the MTw devices from the Awinda station and wait until the devices are connected (synced leds), then press enter... \n')

    # Vérifier les dispositifs détectées (nombre de capteurs MTw)
    children = device.children()

    devIdAll = []
    for i in range(len(children)):
        devIdAll.append(children[i].deviceId()) 

    devicesUsed,devIdUsed,nDevs = checkConnectedSensors(devIdAll)

    ##################

    # Mis les capteurs MTw en mode acquisition des données
    print("Putting devices into measurement mode...\n")
    
    if not device.gotoMeasurement():
        raise RuntimeError("Could not put device into measurement mode. Aborting.")
    print("Creating a log file...")
    logFileName = "logfile.mtb"
    if device.createLogFile(logFileName) != xda.XRV_OK:
        raise RuntimeError("Failed to create a log file. Aborting.")
    else:
        print("Created a log file: %s" % logFileName)
    
    print("Starting recording...")
    
    if not device.startRecording():
        raise RuntimeError("Failed to start recording. Aborting.")

    ### Initialisation of list containing datapackets of MTw ##
    matrix_nDev = list(range(nDevs))
    acc_nDev = list(range(nDevs))
    pack_nDev = list(range(nDevs))
    contCalData_nDev = list(range(nDevs))
    frame_nDev = list(range(nDevs))
    counter_nDev = list(range(nDevs))
    tim_nDev = list(range(nDevs))
    threads = list(range(nDevs))

    for i in range(nDevs):
        pack_nDev[i] = []
        contCalData_nDev[i] = []
        tim_nDev[i] = []
        frame_nDev[i] = []
        counter_nDev[i] = []
        acc_nDev[i] = []
        matrix_nDev[i] = []
        threads[i] = []
    ############################################################
        
    print("Wait 10 seconds before starting data acquisition.\n")
    time.sleep(10)

    input("Press enter to start recording...\n")
    print("Starting acquisition...\n")
    
    
    
    

    keep_going = True

    th.Thread(target=key_capture_thread, args=(), name='key_capture_thread', daemon=True).start()

    for i in range(nDevs):
        threads[i] = th.Thread(target=childCount, args=(i,))
        threads[i].start()

    #time.sleep(1/fq_support[index_fq])
    for i in range(nDevs):
        threads[i].join()
    #####
    for i in range(nDevs):
        print(devIdAll[i],"number of data packet: ",len(counter_nDev[i]))

    ##################

    exit = int(input('\n Appuyez sur la touche 0 pour quitter le mode acquisition... \n'))
    if exit == 0:
        stopAll()
    
except RuntimeError as error:
    print(error)
    sys.exit(1)

except:
    print("An unknown fatal error has occured. Aborting.")
    sys.exit(1)
    
else:
    print("Successful exit.")

Creation d'un objet XsControl...
Utilisation de la version XDA: 2021.0.0 build 5908 rev 109668 built on 2021-04-19 

Détection des ports#...

 Dispositif trouvé avec les caractéristiques suivantes:
 Device ID: 01200816
 Nom du port: COM3
 Port baudrate: 2000000
Ouverture du port...
Dispositif: AW-A2, avec ID: 01200816 ouvert. 

Dispositif en mode configuration...

Les fréquences disponibles sont:
0: 120
1: 100
2: 80
3: 60
4: 40
Fréquence d'acquisition choisi:  120 

Les canaux disponibles sont: 
0: 11
1: 12
2: 13
3: 14
4: 15
5: 16
6: 17
7: 18
8: 19
9: 20
10: 21
11: 22
12: 23
13: 24
14: 25
Canal de radio choisi:  11 

Dispositifs rejetés:
Dispositifs acceptés:
0 - 00B43CC1
Putting devices into measurement mode...



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Creating a log file...
Created a log file: logfile.mtb
Starting recording...
Failed to start recording. Aborting.
Traceback (most recent call last):
  File "<ipython-input-3-40725fe97446>", line 117, in <module>
    raise RuntimeError("Failed to start recording. Aborting.")
RuntimeError: Failed to start recording. Aborting.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\rena_\anaconda3\envs\IMU\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-40725fe97446>", line 173, in <module>
    sys.exit(1)
SystemExit: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\rena_\anaconda3\envs\IMU\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:

TypeError: object of type 'NoneType' has no len()